# Entrenamiento de modelos locales en paralelo con Spark

* Utilizaremos la idea de MapReduce (divide y venceras) para entrenar modelos independientes de acuerdo a las particiones de datos que tengamos.

* Una vez entrenados, tenemos que combinar sus predicciones para obtener una única respuesta.

* Utilizaremos un árbol de decisión de la librería "sci-kit learn" pero en principio podemos ocupar casi cualquier "eager learner".

* Básicamente crearemos un "tree ensemble".

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
from pyspark.sql.types import *

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import accuracy_score


spark = SparkSession \
    .builder \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#--Definimos un esquema para el csv que vamos a leer
#-Sino definimos el esquema Spark lo va inferir con la primera línea,
#-lo que puede no ser la mejor opción. Existe un parámetro llamado
#-"samplingRatio" que utiliza una porción de los datos para inferir el esquema
#-pero puede ser muy costoso con grandes datos.

schema = StructType(
    [
        StructField("Ex01", FloatType(), True),
        StructField("Ex02", FloatType(), True),
        StructField("Ex03", FloatType(), True),
        StructField("Ex04", FloatType(), True),
        StructField("Project", FloatType(), True),
        StructField("Question 1", FloatType(), True),
        StructField("Question 2", FloatType(), True),
        StructField("Question 3", FloatType(), True),
        StructField("Question 4", FloatType(), True),
        StructField("Exam", FloatType(), True),
        StructField("Total", FloatType(), True),
    ]
)

In [ ]:
#--Data set que vamos a leer
INPUT_PATH = '/content/drive/MyDrive/data_sets/marks.csv'

In [ ]:
#--leemos el archivo CSV
df = spark.read.format('csv')\
    .option("header", 'true')\
    .schema(schema)\
    .load(INPUT_PATH)

#--creamos una variable respuesta binaria
df = df.withColumn('response',
                   F.when(df.Total<70, 'not-first')\
                        .otherwise('first'))

#--eliminamos las columnas que no necesitamos
df = df.drop("Question 1")\
       .drop("Question 2")\
       .drop("Question 3")\
       .drop("Question 4")\
       .drop("Exam")\
       .drop("Total")

In [ ]:
df.show(2)

In [ ]:
#--dividimos los datos en conjunto de entrenamiento y prueba
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

El número de modelos que vamos a entrenar dependerá del número de particiones que tengamos en nuestros datos.

Mayor número de particiones, mayor paralelización pero menor cantidad de datos para cada modelo.

In [ ]:
#--En nuestro caso al ser un data set pequeño, vamos a dividir
#-el train set en 3 partes y el de validación en dos partes
10, 30, 100

train_rdd = train_df.rdd.repartition(3)
test_rdd  = test_df.rdd.repartition(2)

El nombre de las columnas será utilizado para crear pandas dataFrames que sean homogeneos dentro de cada "worker" por lo que esta variable debe estar disponible para todos, debe ser una variable global. Si trabajamos con "notebooks" basta asignar el nombre de las columnas a una variable

In [ ]:
column_names = df.columns

## Entrenamiento del modelo en paralelo

Definimos una función que será la encargada de hacer el entrenamiento. Está función se mandará a todos los "workers" y cada uno entrenara un modelo de forma independiente.

El resultado sera una lista de modelos entrenados con diferentes particiones de la data

In [ ]:
def build_model(partition_data_it):

    #-Tenemos que transformar la data a pandas
    partition_data_df = pd.DataFrame(partition_data_it, columns=column_names)

    #-Definimos el modelo que deseamos ocupar
    clf = DecisionTreeClassifier(random_state=0, max_depth=2)

    #-Dividimos los datos en atributos y variable respuesta
    X_train = partition_data_df.iloc[:, :-1]
    y_train = partition_data_df["response"]

    #-Entrenamos el modelo
    model = clf.fit(X_train.values, y_train.values)

    return [model]

Utilizamos "mapPartitions" para entrenar nuestros modelos en lugar del clásico "fit".

__mapPartitions__: aplica una función a cada partición, la función recibe un "iterador" y regresa otro "iterador".

In [ ]:
#--realizamos el entrenamiento y traemos todos los modelos al node "driver"
models = train_rdd.mapPartitions(build_model).collect()

print(f'Número de modelos entrenados: {len(models)}')

## Realizar predicciones en paralelo

En este ejemplo tenemos 3 modelos por lo que cada uno de ellos deberá hacer una predicción para cada una de las muestras y al final debemos combinar las predicciones para obtener una predicción por muestra.

Los modelos están en el "driver", para hacer las predicciones debemos mandarlos a los "workers". En este ejemplo estamos usando los modelos como una variable global debido a que son "pequeños". Si los modelos son grandes entonces debemos usar un "broadcasting".

In [ ]:
#--Definimos una función que va a tomar una muestra (fila de nuestro data set)
#-y va a regresar una lista con las predicciones de cada modelo
def predict(instance):
    X = instance[:-1]
    return [m.predict([X])[0] for m in models]

In [ ]:
#--probamos la función y vemos la respuesta de las primeras n filas
test_rdd.map(predict).take(3)

Necesitamos definir una forma de combinar las salidas de cada modelo para dar una respuesta única por muestra o instancia.

La forma más sencilla en clasificación, y la forma que vamos a explorar en este notebook, es por mayoría de votos.

Otras formas pueden ser un promedio de las probabilidades o promedios ponderados.

In [ ]:
#--definimos la función que hará el trabajo de combinar las respuestas.
#-La estrategía será definir un diccionario y contar cuantos votos hay de
#-cada clase y elegir la clase con más votos como respuesta final

def agg_predictions(preds):
    #--definimos el diccionario
    predictions = { "first": 0, "not-first" : 0 }

    #--contamos los elementos
    for elem in preds:
        predictions[elem]+= 1

    #--regresamos solamente la llave del valor máximo
    return max(predictions, key=predictions.get)

In [ ]:
#--probamos la función anterior
test_rdd.map(predict).map(agg_predictions).take(5)

In [ ]:
#--definimos una función que encapsule las dos funciones anteriores y
#-de formato a los resultados tal que podamos tenerlos en un spark dataFrame:
#-unir los atributos, la variable respuesta real y la predicción en una sola
#-fila

def transform(instance):
    return Row(**instance.asDict(),\
               raw_prediction=agg_predictions(predict(instance)))

In [ ]:
#--hacemos las predicciones
prediction = test_rdd.map(transform).toDF()
prediction.show(2)

### Evaluación de modelo

In [ ]:
#--transformar las predicciones de string a númericas {0,1}
prediction_num = prediction.select(
    (prediction["response"] == "first").cast("double").alias("label"),
    (prediction["raw_prediction"] == "first").cast("double").alias("pred"),
)

acc_evaluator = MulticlassClassificationEvaluator(
    metricName="accuracy",
    labelCol="label",
    predictionCol="pred"
)

print(f'Accuracy: {acc_evaluator.evaluate(prediction_num):.3f}')

## Comparación con un solo modelo de sk-learn

In [ ]:
#--convertimos los datos a pandas
train_pd = train_df.toPandas()
test_pd = test_df.toPandas()

In [ ]:
#--definimos el modelos
clf = DecisionTreeClassifier(random_state=0, max_depth=2)

#--dividimos los atributos y la variable respuesta
X_train = train_pd.iloc[:, :-1]
y_train = train_pd["response"]

#--entrenamos el clasificador
clf.fit(X_train, y_train)

In [ ]:
#--separamos atributos y variable respuesta del test set
X_test = test_pd.iloc[:, :-1]

#--realizamos la predicciones
y_pred = clf.predict(X_test)

#--medimos la precisión
y_test = test_pd['response']
print(f'Accuracy sk-learn model: {accuracy_score(y_test, y_pred):.3f}')